In [ ]:
import gzip
from collections import defaultdict
from datetime import datetime
from datetime import datetime
from tqdm import tqdm
import random
import pandas as pd

### 使用データセット

In [ ]:
# 　使用データセット
dataset_name = 'Beauty_5'

### item/user の設定

In [ ]:
# item/userの固定フラグ
is_item_per_user_num_const = True
item_per_user = 5

In [ ]:
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield eval(l)

In [ ]:
countU = defaultdict(lambda: 0)
countP = defaultdict(lambda: 0)
line = 0

In [ ]:
f = open('data/reviews_' + dataset_name + '.txt', 'w')

In [ ]:
for l in parse('data/reviews_' + dataset_name + '.json.gz'):
    line += 1
    f.write(" ".join([l['reviewerID'], l['asin'], str(l['overall']), str(l['unixReviewTime'])]) + ' \n')
    asin = l['asin']
    rev = l['reviewerID']
    time = l['unixReviewTime']
    countU[rev] += 1
    countP[asin] += 1
f.close()

In [ ]:
users = []
items = []
times = []

for l in parse('data/reviews_' + dataset_name + '.json.gz'):
    line += 1
    items.append(l['asin'])
    users.append(l['reviewerID'])
    times.append(l['unixReviewTime'])

item/userを一定に整形

In [ ]:
if is_item_per_user_num_const:
    print('is_item_per_user_num_const = {}'.format(is_item_per_user_num_const))
    print('item/user = {}'.format(item_per_user))
    
    User_all = defaultdict(list)
    User = defaultdict(list)
    
    for u, i, t in zip(users, items, times):
        User_all[u].append([i,t])
        
    for u in User_all:
        items = User_all[u]
        if len(items) > item_per_user:
            items = items[0:item_per_user]
            User[u] = items
    
    users = []
    items = []
    times = []
    
    for u in User:
        for values in User[u]:
            users.append(u)
            items.append(values[0])
            times.append(values[1])

In [ ]:
print('item/user > {} を満たすユーザ数:{}'.format(item_per_user, len(set(users))))

### user数 の設定

In [ ]:
# ユーザ数の指定フラグ
is_user_num_const = True
user_num = 30

指定したユーザ数にデータを整形

In [ ]:
if is_user_num_const:
    print('is_user_num_const = {}'.format(is_user_num_const))
    
    unique_users = set(users)
    master_user = random.sample(list(unique_users), user_num)
    
    users_ = users
    items_ = items
    times_ = times
    
    users = []
    items = []
    times = []
    
    for u, i, t in tqdm(zip(users_, items_, times_)):
        if u in master_user:
            users.append(u)
            items.append(i)
            times.append(t)

In [ ]:
usermap = dict()
usernum = 0
itemmap = dict()
itemnum = 0
User = dict()

In [ ]:
for user, item, time in zip(users, items, times):
    #  評価件数がitem_per_user以下のものは無視    
    if countU[user] < item_per_user :
        continue
        
    if user in usermap:
        userid = usermap[user]
    else:
        usernum +=1
        userid = usernum
        usermap[user] = userid
        User[userid] = []
        
    if item in itemmap:
        itemid = itemmap[item]
    else:
        itemnum += 1
        itemid = itemnum
        itemmap[item] = itemid
        
    User[userid].append([time, itemid])

In [ ]:
# 時系列順にソートする
for userid in User.keys():
    User[userid].sort(key=lambda x: x[0])

In [ ]:
print (usernum, itemnum)

In [ ]:
usermap
user_ids = list(usermap.keys())
fixed_user_ids = list(usermap.values())

In [ ]:
df_user_ids = pd.DataFrame(
    data={
        'user_ids' : user_ids,
        'fixed_user_ids' : fixed_user_ids
    },
    columns=['user_ids', 'fixed_user_ids']
)

In [ ]:
itemmap
item_ids = list(itemmap.keys())
fixed_item_ids = list(itemmap.values())

In [ ]:
df_movie_ids = pd.DataFrame(
    data={
        'item_ids' : item_ids,
        'fixed_item_ids' : fixed_item_ids
    },
    columns=['item_ids', 'fixed_item_ids']
)

In [ ]:
each_item_num = '_ItemPerUser=' + str(item_per_user) if is_item_per_user_num_const else ''
user_num_in_data= '_UserNum=' + str(user_num) if is_user_num_const else ''

In [ ]:
f = open('data/processing_data/{}_processing{}{}.txt'.format(dataset_name,user_num_in_data, each_item_num), 'w')
for user in User.keys():
    for i in User[user]:
        f.write('%d %d\n' % (user, i[1]))
f.close()

In [ ]:
df_user_ids.to_csv('data/processing_data/{}_usermap{}{}.csv'.format(dataset_name,user_num_in_data, each_item_num))
df_movie_ids.to_csv('data/processing_data/{}_itemmap{}{}.csv'.format(dataset_name,user_num_in_data, each_item_num))